In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from io import StringIO
from collections import Counter
from Bio import pairwise2
import subprocess
from pyvis.network import Network

def preprocess_reads(file_path):
    """Load file and remove headers."""
    with open(file_path, "r") as file:
        lines = file.readlines()
    cleaned_reads = [line.strip() for line in lines if not line.startswith('>') and line.strip()]
    return cleaned_reads

def classify_reads_by_length(reads, threshold=1000):
    short_reads = [read for read in reads if len(read) < threshold]
    long_reads = [read for read in reads if len(read) >= threshold]
    return short_reads, long_reads

def dynamic_kmer_selection(reads):
    avg_length = np.mean([len(read) for read in reads])
    return max(15, min(31, int(avg_length / 2)))

def generate_kmers(reads, k):
    return [read[i:i+k] for read in reads for i in range(len(read) - k + 1)]

def filter_low_frequency_kmers(kmers, threshold=2):
    kmer_counts = Counter(kmers)
    return [k for k in kmers if kmer_counts[k] >= threshold]

def correct_errors_with_kmers(kmers):
    kmer_counts = Counter(kmers)
    return [k for k in kmers if kmer_counts[k] > 1]

def construct_de_bruijn_graph(kmers):
    G = nx.DiGraph()
    edges = [(kmer[:-1], kmer[1:]) for kmer in kmers]
    G.add_edges_from(edges)
    return G

def prune_graph(G, min_degree=2):
    nodes_to_remove = [node for node, degree in G.degree() if degree < min_degree]
    G.remove_nodes_from(nodes_to_remove)
    return G

def simplify_graph(G):
    largest_cc = max(nx.weakly_connected_components(G), key=len, default=set())
    return G.subgraph(largest_cc).copy()

def find_eulerian_path(G):
    return list(nx.eulerian_path(G)) if nx.is_eulerian(G) else []

def scaffold_with_long_reads(assembled_sequence, long_reads):
    scaffolds = assembled_sequence.copy()
    for long_read in long_reads:
        i = 0
        while i < len(scaffolds) - 1:
            if scaffolds[i] in long_read and scaffolds[i+1] in long_read:
                scaffolds[i] += "N" * 10 + scaffolds[i+1]
                scaffolds.pop(i+1)
            else:
                i += 1
    return scaffolds

def hybrid_scaffolding(assembled_sequence, long_reads, short_reads):
    long_scaffolds = scaffold_with_long_reads(assembled_sequence, long_reads)
    return long_scaffolds  # Placeholder for short read scaffolding

def visualize_graph(G):
    plt.figure(figsize=(10, 6))
    pos = nx.spring_layout(G, seed=42)
    nx.draw(G, pos, with_labels=True, node_size=100, font_size=8, edge_color='gray')
    plt.show()

def save_result(sequence, filename="assembled_genome.txt"):
    with open(filename, "w") as f:
        f.write("\n".join(sequence))
    print(f"Assembled genome saved to {filename}")

def check_errors(assembled_sequence, reference_genome):
    alignments = pairwise2.align.globalxx(assembled_sequence, reference_genome)
    score = alignments[0][2] if alignments else 0
    if score < 0.9 * len(assembled_sequence):
        print("Warning: Assembly might contain errors, score:", score)
    else:
        print("Success: Assembly appears to be error-free.")

def run_bwa_alignment(assembled_sequence, reference_genome):
    with open("assembled_sequence.fasta", "w") as f:
        f.write("\n".join(assembled_sequence))

    with open("reference_genome.fasta", "w") as f:
        f.write(reference_genome)

    subprocess.run(["bwa", "mem", "reference_genome.fasta", "assembled_sequence.fasta", "-o", "alignment.sam"])

def run_genome_assembly(reads_file, reference_file):
    reads = preprocess_reads(reads_file)
    short_reads, long_reads = classify_reads_by_length(reads)
    k = dynamic_kmer_selection(short_reads + long_reads)
    print(f"Using k-mer size: {k}")
    kmers = generate_kmers(short_reads + long_reads, k)
    kmers = filter_low_frequency_kmers(kmers)
    kmers = correct_errors_with_kmers(kmers)
    G = construct_de_bruijn_graph(kmers)
    G = prune_graph(G)
    G = simplify_graph(G)
    assembled_sequence = find_eulerian_path(G)
    assembled_sequence = [edge[0] for edge in assembled_sequence]
    scaffolds = hybrid_scaffolding(assembled_sequence, long_reads, short_reads)
    visualize_graph(G)
    save_result(scaffolds)
    with open(reference_file, "r") as ref_file:
        reference_genome = ref_file.read().replace("\n", "")
    check_errors(scaffolds, reference_genome)
    run_bwa_alignment(scaffolds, reference_genome)

# Example usage:
run_genome_assembly("/content/SRR31302084.fasta", "/content/SRR31302084.fasta")
